In [1]:
import openai
import re
import time
import numpy as np

from tqdm import tqdm
from pprint import pprint


from agents import SCAgent, initial_instructions, CounselorAgent, PatientAgent, ModeratorAgent, wprint
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [2]:
#Note: The openai-python library support for Azure OpenAI is in preview.
#Note: This code sample requires OpenAI Python library version 0.28.1 or lower.

import openai
import os

openai.api_type = "azure"
openai.api_base = "https://tfccgpoc.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

message_text = [{"role":"system","content":"you are a expert AI system"},{"role":"user","content":" what is openAI?"}]

completion = openai.ChatCompletion.create(
  engine="TFccgUsecase",
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion['choices'][0]['message'])

{
  "role": "assistant",
  "content": "OpenAI is an artificial intelligence research laboratory consisting of the for-profit corporation OpenAI LP and its parent company, the non-profit OpenAI Inc. The organization is dedicated to advancing AI in a safe and beneficial way for humanity. OpenAI was founded in December 2015 by a group of technology leaders, including Elon Musk and Sam Altman, with the goal of developing AI systems that are safe, beneficial, and aligned with human values. Since then, OpenAI has made significant advancements in the field of AI, including developing AI models for natural language processing, robotics, and more."
}


***Counselor Init***

In [3]:
counselor_initial_dialog_history = initial_instructions('Genetic_counselor.txt')
counselor = CounselorAgent(initial_dialog_history=counselor_initial_dialog_history, agent_type="counselor", engine="gpt-3.5-turbo")
pprint(counselor.dialog_history)

[{'content': 'you are a expert genetic counsellor . please ask one question '
             'and wait for patient to answer before asking next question. '
             'cover all questions about family history, patient history and '
             'pathology report. \n'
             'after all questions are done explain how genetic testing works. '
             'your goal is to provide clear and relevant information about '
             'genetic risk factors in a supportive and educational '
             'environment. at the very end offer to schedule a appointment',
  'role': 'system'},
 {'content': 'Counseling the patient', 'role': 'user'}]


***Patient Init***

In [4]:
patient_initial_dialog_history = initial_instructions('Patient.txt')
patient = PatientAgent(initial_dialog_history=patient_initial_dialog_history, agent_type="patient", engine="gpt-3.5-turbo")
pprint(patient.dialog_history)

[{'content': 'You are a patient chatting with a doctor over an online chat '
             'interface. The counselor has never met you before. \n'
             'your background is you are  a 58-year-old African American male, '
             'presents with complaints of bone pain and difficulty urinating. '
             'He has a history of diabetes and is a former smoker.\n'
             'Prostate biopsy demonstrated a Gleason score of 9 (4+5), '
             'involving 70% of the total core length. There is evidence of '
             "perineural invasion. The patient's clinical stage is T3a.\n"
             'His brother was diagnosed with metastatic prostate cancer at the '
             'age of 60.',
  'role': 'system'},
 {'content': 'patient', 'role': 'user'}]


***Moderator Init***

In [5]:
moderator_initial_dialog_history = initial_instructions("Moderator.txt")
moderator = ModeratorAgent(initial_dialog_history=moderator_initial_dialog_history, agent_type="moderator", engine="gpt-3.5-turbo")
pprint(moderator.dialog_history)

[{'content': 'Now enter the role-playing mode. In the following conversation, '
             'you will play as a moderator for a dialog.',
  'role': 'system'},
 {'content': 'In the following conversation, you will play as a moderator in a '
             'discussion between a genetic counselor and a patient. you have '
             'to evaluate if either of them are asking a question. if yes then '
             'the conversation is not completed. \n'
             'The conversation is complete if the counselor offers a '
             'appointment or if patient discusses a appointment\n'
             'conversation is also complete if counselor has offered details '
             'of genetic testing\n'
             'conversation is also complete if patient says thank you',
  'role': 'user'}]


***Complete conversation + test selection***

In [17]:
def run(counselor, patient, moderator, n=6, fd=None, verbose=False):
    wprint('counselor: %s' % counselor.last_response, fd, verbose=verbose)
    wprint('patient: %s' % patient.last_response, fd, verbose=verbose)
    
    wprint('---- start conversation ----', fd, verbose=verbose)
    patient_run = patient.last_response
    start_involve_moderator = False
     
    moderate=''
    for _ in range(n):
        time.sleep(.1)
        counselor_run = counselor.call(patient_run)
        wprint('counselor: %s \n' % counselor.last_response, fd, verbose=verbose)
        
        if(start_involve_moderator is False ):
            start_involve_moderator = True
            #wprint('---- start moderating ----', fd, verbose=verbose)
        
        if(start_involve_moderator):
            time.sleep(.1)
            moderate = moderator.moderate(counselor.dialog_history,who_was_last="counselor")
            #wprint('MODERATER has the conversation concluded? Yes or No: %s' % moderate, fd, verbose=verbose)
            if("yes" in moderate.lower()): 
                
                break
            else: pass

        time.sleep(.1)   
        patient_run = patient.call(counselor_run)
        wprint('patient: %s \n' % patient.last_response, fd, verbose=verbose)
        
        if(start_involve_moderator is False ):
            start_involve_moderator = True
            #wprint('---- start moderating ----', fd, verbose=verbose)
            
        if(start_involve_moderator):
            time.sleep(.1)
            moderate = moderator.moderate( patient.dialog_history, who_was_last="patient")
            #wprint('MODERATER has the conversation concluded? Yes or No: %s' % moderate, fd, verbose=verbose)
            if("yes" in moderate.lower()): 
                
                break
            else: pass
                

    print("=======Moderator=====")
    print('\n %s' % moderate)
    if("yes" in moderate.lower()):
            print("conversation concluded")
            counselor_run = counselor.call("patient: what neogenomics panel is recommended for me \
                please select from following \
                Neotype Lung tumor profile, \
                Neotype Breast tumor profile, \
                Neotype Colon tumor profile, \
                Neotype Gastric tumor profile, \
                Neotype Kidney tumor profile, \
                Neotype Ovarian tumor profile, \
                Neotype Pancreatic tumor profile, \
                Neotype Prostrate tumor profile, \
                Neotype Lynch Syndrome profile, \
                Neotype Uterine tumor profile, \
                     other panel?")
            wprint('counselor: %s' % counselor.last_response, fd, verbose=verbose)
    return counselor.last_response
    

***Actual run***

In [ ]:
patient.reset()
counselor.reset()
moderator.reset()
response = run(counselor, patient, moderator, n=6)

openai.api_type = "azure"
openai.api_base = "https://tfccgpoc.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

prompt = "what is the name of panel in this text, it will start with Neotype word, return as a string name:" + response

message_text = [{"role":"system","content":"you are a average human"},{"role":"user","content":prompt}]

completion = openai.ChatCompletion.create(
  engine="TFccgUsecase",
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion['choices'][0]['message'])
 

***100 sample test***

In [19]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import openpyxl
import os

specific_rows = [0, 1, 11, 21, 31, 41, 51, 61, 71, 81, 91]
patientjson = [{'content': 'You are a patient chatting with a doctor over an online chat '
             'interface. The counselor has never met you before. \n'
             'your background is you are  a 58-year-old African American male, '
             'presents with complaints of bone pain and difficulty urinating. '
             'He has a history of diabetes and is a former smoker.\n'
             'Prostate biopsy demonstrated a Gleason score of 9 (4+5), '
             'involving 70% of the total core length. There is evidence of '
             "perineural invasion. The patient's clinical stage is T3a.\n"
             'His brother was diagnosed with metastatic prostate cancer at the '
             'age of 60.',
  'role': 'system'},
 {'content': 'patient', 'role': 'user'}]


df = pd.read_excel('PatientVignettes.xlsx',skiprows = lambda x:x not in specific_rows)

for i, row in df.iterrows():
        val = df.at[i,'Type']
        print(val)

        prompt = " You are a patient chatting with a doctor over an online chat interface. The counselor has never met you before. \
                your background is you are "

        patientjson[0]['content']= prompt + os.linesep\
                  + df.at[i,'Demographics'] + os.linesep +df.at[i,'Pathology'] + os.linesep\
                         + df.at[i,'Family History'] 
        
        patient_initial_dialog_history = patientjson
        patient = PatientAgent(initial_dialog_history=patient_initial_dialog_history, agent_type="patient", engine="gpt-3.5-turbo")
        #pprint(patient.dialog_history)

        patient.reset()
        counselor.reset()
        moderator.reset()
        response = run(counselor, patient, moderator, n=6)

        openai.api_type = "azure"
        openai.api_base = "https://tfccgpoc.openai.azure.com/"
        openai.api_version = "2023-09-15-preview"
        openai.api_key = os.getenv("OPENAI_API_KEY")

        prompt = "what is the name of panel in this text, it will start with Neotype word, return as a string name:" + response

        message_text = [{"role":"system","content":"you are a average human"},{"role":"user","content":prompt}]

        completion = openai.ChatCompletion.create(
        engine="TFccgUsecase",
        messages = message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )

        print(completion['choices'][0]['message'])
     

Lung 
=======Moderator=====

 Yes.
conversation concluded
{
  "role": "assistant",
  "content": "Neotype Lung tumor profile"
}
Breast
=======Moderator=====

 Yes, the conversation has reached a conclusion as the patient has requested assistance in scheduling a genetic counseling appointment and the counselor has offered to help and provided resources.
conversation concluded
{
  "role": "assistant",
  "content": "Neotype tumor profiles"
}
Colon
=======Moderator=====

 Yes. The conversation has reached a conclusion as the patient has requested to schedule an appointment with a genetic counselor and thanked the counselor for their guidance.
conversation concluded
{
  "role": "assistant",
  "content": "Neotype Colon Tumor Profile"
}
Gastric
=======Moderator=====

 Yes, the conversation has reached a conclusion as the patient has expressed interest in pursuing genetic testing and the counselor has offered a referral to a genetic counselor. Additionally, the patient has said thank you, indic